In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

from openpyxl import load_workbook
import operator
import pickle
import time
import math

In [2]:
# Read coefficients of model
in_stream = open("compact_merge_coef.pkl", "rb")
dfCoef = pickle.load(in_stream)
in_stream.close()

df = pd.DataFrame.from_csv('brand_features_singleword.csv')
intercept = 0.04467419
L = ['BRAND']
L.extend(dfCoef[0])

dfFeatures = df[L]

In [4]:
sheet = 1
count = 0
for category in df.CATEGORY.unique():

    
    print(category)
    
    num_females = 0
    num_total = 0
    
    dfCurrent = dfFeatures.loc[df['CATEGORY'] == category]
    
    name_predictions = [("Name", "Prediction", "Male Confidence", "Female Confidence")]
    male_confidence = []
    female_confidence = []
    
    for i in range(len(dfCurrent)):
        dot = 0
        for f in range(len(dfCoef[1])):
            dot += dfCurrent.iloc[i][f+1] * dfCoef[1][f]
        prediction_probability = 1 / (math.exp(- dot - intercept) + 1)
        
        num_total += 1
        if prediction_probability >= 0.5:
            num_females += 1
            name_predictions.append((dfCurrent.iloc[i][0], prediction_probability, None, prediction_probability - 0.5))
            female_confidence.append(prediction_probability - 0.5)
        else:
            name_predictions.append((dfCurrent.iloc[i][0], prediction_probability, abs(prediction_probability - 0.5), None))
            male_confidence.append(abs(prediction_probability - 0.5))
    
    female_ratio = num_females / num_total
    
    name_predictions[0] = ("Name", "Prediction", "Male Confidence", "Female Confidence", "Female Ratio: " + str(female_ratio))
    
    print("Female ratio: " + str(female_ratio))
    
    dfPredict = pd.DataFrame(name_predictions)
    excel_dir = 'brand_predictions.xlsx'
    book = load_workbook(excel_dir)
    with pd.ExcelWriter(excel_dir, engine='openpyxl') as writer:
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)    

        ## Your dataframe to append. 
        dfPredict.to_excel(writer, category, index=False)

        writer.save()  
    writer.close()
    
    count += num_total
    sheet += 1
    
    file_name = category

    bar_nums = [1,2]
    ratios = [1 - female_ratio, female_ratio]

    LABELS = ["Male", "Female"]

    plt.subplot(1, 3, 1)
    plt.bar(bar_nums, ratios, align='center')
    plt.xticks(bar_nums, LABELS)

    plt.title(file_name + '\nGender Ratio (N=' + str(num_total) + ')')
    plt.ylabel("Ratio (%)")
    plt.xlabel("Gender")
    
    for i,j in zip(bar_nums,ratios):
        plt.annotate(str(round(j, 2)),xy=(i,j))
    plt.subplot(1, 2, 2)
    
    data = [male_confidence, female_confidence]
#     data = np.concatenate((male_confidence, female_confidence), 0)
    plt.boxplot(data, labels = ['Male', 'Female'])
    plt.ylim([0, 0.5])
    plt.ylabel("Confidence")
#     plt.boxplot(male_confidence, positions = [1, 2])

#     plt.boxplot(female_confidence, positions = [3, 4])
    
    
    plt.savefig(file_name + '.png')
    plt.clf()
    
    
    
    
print("Total: " + str(count))

ANALGESICS
Female ratio: 0.40540540540540543
BABY_FOODS
Female ratio: 0.7777777777777778
BASEBALL_EQUIPMENT
Female ratio: 0.26666666666666666
CARS_TRUCKS
Female ratio: 0.42857142857142855
COSMETICS
Female ratio: 0.5041322314049587
FINANCIAL_SERVICES
Female ratio: 0.21739130434782608
BBQ
Female ratio: 0.3333333333333333
HAIR_PRODUCTS
Female ratio: 0.3684210526315789
JEWELRY
Female ratio: 0.4377104377104377
MEN_HAIRCARE
Female ratio: 0.5
MEN_UNDERWEAR
Female ratio: 0.4117647058823529
MILITARY
Female ratio: 0.4166666666666667
POWER_TOOLS
Female ratio: 0.3229166666666667
TELEVISIONS
Female ratio: 0.4583333333333333
WATCHES
Female ratio: 0.30303030303030304
WOMENS_UNDERWEAR
Female ratio: 0.6
YOGURT
Female ratio: 0.375
Total: 1087


In [220]:
dfPredict.to_csv('Brand Predictions.csv')

In [205]:
file_name = 'example'

DayOfWeekOfCall = [1,2]
DispatchesOnThisWeekday = [0.3, 0.7]

LABELS = ["Male", "Female"]

plt.bar(DayOfWeekOfCall, DispatchesOnThisWeekday, align='center')
plt.xticks(DayOfWeekOfCall, LABELS)

plt.title(file_name + ' Gender Distribution')
plt.ylabel("Ratio (%)")
plt.xlabel("Gender")

plt.savefig(file_name + '.png')


In [5]:
dfCSV = pd.DataFrame.from_csv("brand_features_singleword.csv")

In [8]:
dfCSV[["BRAND","CATEGORY", "DOLS"]].to_csv("brand_singleword.csv")